In [1]:
# https://github.com/pltrdy/rouge
from rouge import Rouge

# https://github.com/li-plus/rouge-metric
from rouge_metric import PyRouge

# https://huggingface.co/metrics/rouge
from datasets import load_metric


In [2]:
# Example Input
preds=["Model output 1","Model output 2","Model output 3"]

# Single Reference for each sample
references=["Reference 1","Reference 2","Reference 3"]

# Multiple reference for each sample
multiple_references=[
	["Sample 1 Reference 1","Sample 1 Reference 2","Sample 1 Reference 3"],
	["Sample 2 Reference 1","Sample 2 Reference 2","Sample 2 Reference 3"],
	["Sample 3 Reference 1","Sample 3 Reference 2","Sample 3 Reference 3"]
]

In [3]:
rouge = Rouge()
scores=rouge.get_scores(preds, references, avg=True)
print(scores)

{'rouge-1': {'r': 0.5, 'p': 0.3333333333333333, 'f': 0.3999999952000001}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.5, 'p': 0.3333333333333333, 'f': 0.3999999952000001}}


In [7]:
# Evaluate document-wise ROUGE scores
# skip_gap: The maximum gap between two words in skip-bigram
# mode: 'average', 'individual'
rouge = PyRouge(rouge_n=(1, 2, 4), rouge_l=True, rouge_w=True,
                rouge_w_weight=1.2, rouge_s=True, rouge_su=True#, skip_gap=4
				,mode='average')
scores=scores = rouge.evaluate(preds, references)
print(scores)

{'rouge-1': {'r': 0.10000000000000002, 'p': 0.030303030303030304, 'f': 0.04651162790697676}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-4': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.10000000000000002, 'p': 0.030303030303030304, 'f': 0.04651162790697676}, 'rouge-w-1.2': {'r': 0.14677992676220694, 'p': 0.045190953800397234, 'f': 0.06910553173165694}, 'rouge-s*': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-su*': {'r': 0.0, 'p': 0.0, 'f': 0.0}}


In [6]:
metric = load_metric("rouge")
scores=metric.compute(predictions=preds, references=references, rouge_types=["rougeL"])
print(scores)

{'rougeL': AggregateScore(low=Score(precision=0.3333333333333333, recall=0.5, fmeasure=0.4000000000000001), mid=Score(precision=0.3333333333333333, recall=0.5, fmeasure=0.4000000000000001), high=Score(precision=0.3333333333333333, recall=0.5, fmeasure=0.4000000000000001))}
